In [ ]:
from DistMLIP.implementations.matgl import TensorNet_Dist, Potential_Dist, MolecularDynamics, Relaxer
import matgl
from pymatgen.core import Structure, Lattice
from pymatgen.io.ase import AseAtomsAdaptor

In [ ]:
# Initialize a CHGNet_Dist model (because CHGNet_Dist inherits from matgl's CHGNet class, model finetuning, training, etc. all work the exact same as in MatGL)
# However, we recommend using DistMLIP purely as an inference platform
model = CHGNet_Dist()

# Or load a previously trained CHGNet model
model = matgl.load_model("TensorNet-MatPES-PBE-v2025.1-PES").model

# Make a distributed version of the model
dist_model = TensorNet_Dist.from_existing(model)

# Enable distributed mode for 2 GPUs
dist_model.enable_distributed_mode(["cpu", "cpu"])

In [ ]:
# Insert your atoms
struct = Structure.from_spacegroup("Pm-3m", Lattice.cubic(3.5), ["Li", "Mn"], [[0, 0, 0], [0.5, 0.5, 0.5]])
struct.perturb(1)
struct.make_supercell((13, 7, 7))
print(f"There are {len(struct)} atoms.")
atoms = AseAtomsAdaptor().get_atoms(struct)

In [ ]:
# Create Potential_Dist object, use 128 threads when creating graph structures
potential = Potential_Dist(model=dist_model, num_threads=128, calc_stresses=True)

In [ ]:
# Perform static point calculation
results = potential(atoms)

In [ ]:
from matgl.apps.pes import Potential
from matgl.ext.ase import PESCalculator
model = matgl.load_model("TensorNet-MatPES-PBE-v2025.1-PES").model
base_pes = Potential(model)
base_calc = PESCalculator(base_pes)

In [ ]:
base_calc.calculate(atoms=atoms)

In [ ]:
base_calc.results['energy'], results[0]

In [ ]:
base_calc.results['forces'], results[1]